<a href="https://colab.research.google.com/github/allenporter/home-assistant-datasets/blob/main/train/llama31_unsloth.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# LLama3.1 8b fine tuning

Using the [LLama-3.1 8B Unsloth](https://colab.research.google.com/drive/1Ys44kVvmeZtnICzWz0xgpRnrIOjZAuxp?usp=sharing#scrollTo=LjY75GoYUCB8) notebook

In [1]:
%%capture
# Installs Unsloth, Xformers (Flash Attention) and all other packages!
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
!pip install --no-deps "xformers<0.0.27" "trl<0.9.0" peft accelerate bitsandbytes

# Unsloth

In [2]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

# 4bit pre quantized models we support for 4x faster downloading + no OOMs.
fourbit_models = [
    "unsloth/Meta-Llama-3.1-8B-bnb-4bit",      # Llama-3.1 15 trillion tokens model 2x faster!
    "unsloth/Meta-Llama-3.1-8B-Instruct-bnb-4bit",
    "unsloth/Meta-Llama-3.1-70B-bnb-4bit",
    "unsloth/Meta-Llama-3.1-405B-bnb-4bit",    # We also uploaded 4bit for 405b!
    "unsloth/Mistral-Nemo-Base-2407-bnb-4bit", # New Mistral 12b 2x faster!
    "unsloth/Mistral-Nemo-Instruct-2407-bnb-4bit",
    "unsloth/mistral-7b-v0.3-bnb-4bit",        # Mistral v3 2x faster!
    "unsloth/mistral-7b-instruct-v0.3-bnb-4bit",
    "unsloth/Phi-3-mini-4k-instruct",          # Phi-3 2x faster!d
    "unsloth/Phi-3-medium-4k-instruct",
    "unsloth/gemma-2-9b-bnb-4bit",
    "unsloth/gemma-2-27b-bnb-4bit",            # Gemma 2x faster!
] # More models at https://huggingface.co/unsloth

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Meta-Llama-3.1-8B-Instruct-bnb-4bit",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
==((====))==  Unsloth 2024.8: Fast Llama patching. Transformers = 4.44.0.
   \\   /|    GPU: Tesla T4. Max memory: 14.748 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.3.1+cu121. CUDA = 7.5. CUDA Toolkit = 12.1.
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.26.post1. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/5.70G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/234 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/55.4k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/340 [00:00<?, ?B/s]

In [3]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

Unsloth 2024.8 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


# Training Dataset

In [21]:
%%capture
!git clone https://github.com/allenporter/home-assistant-datasets

In [26]:
import itertools
import json
import random
import pathlib
import math
from datasets import Dataset, Features, Value


DATASET_DIR = pathlib.Path("home-assistant-datasets/datasets/device-actions-v2-collect/train/json/")
NUM_SHARDS = 10

files = list(DATASET_DIR.glob("*.json"))
random.shuffle(files)

def chunk_into_n(lst, n):
  size = math.ceil(len(lst) / n)
  return list(
    map(lambda x: lst[x * size:x * size + size],
    list(range(n)))
  )

shards = list(chunk_into_n(files, NUM_SHARDS))
train_shards = shards[:-1]
eval_shards = shards[-1:]

def gen(shards: list[list[pathlib.Path]]):
  for shard in shards:
    for filename in shard:
      yield json.loads(filename.read_text())

context_feat = Features({'text': Value(dtype='string', id=None)})

train_ds = Dataset.from_generator(gen, gen_kwargs={"shards": train_shards}, features=context_feat, split="train")
eval_ds = Dataset.from_generator(gen, gen_kwargs={"shards": eval_shards}, features=context_feat, split="test") #, streaming=True)

Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

(2357, 10, 1)

# Trainer

In [29]:
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

MAX_STEPS = 10

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = train_ds,
    eval_dataset = eval_ds,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False, # Can make training 5x faster for short sequences.
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        # num_train_epochs = 1, # Set this for 1 full training run.
        max_steps = MAX_STEPS,
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
    ),
)

Map (num_proc=2):   0%|          | 0/2124 [00:00<?, ? examples/s]

Map (num_proc=2):   0%|          | 0/233 [00:00<?, ? examples/s]

max_steps is given, it will override any value given in num_train_epochs


In [30]:
#@title Show current memory stats
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

GPU = Tesla T4. Max memory = 14.748 GB.
5.984 GB of memory reserved.


In [33]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 2,124 | Num Epochs = 1
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 4
\        /    Total batch size = 8 | Total steps = 10
 "-____-"     Number of trainable parameters = 41,943,040


Step,Training Loss
1,0.092800
2,0.117500
3,0.128400
4,0.109600
5,0.091500
6,0.079800


Step,Training Loss
1,0.092800
2,0.117500
3,0.128400
4,0.109600
5,0.091500
6,0.079800
7,0.075400
8,0.065500
9,0.069900
10,0.076500


In [35]:
#@title Show final memory and time stats
used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
used_memory_for_lora = round(used_memory - start_gpu_memory, 3)
used_percentage = round(used_memory         /max_memory*100, 3)
lora_percentage = round(used_memory_for_lora/max_memory*100, 3)
print(f"{trainer_stats.metrics['train_runtime']} seconds used for training.")
print(f"{round(trainer_stats.metrics['train_runtime']/60, 2)} minutes used for training.")
print(f"Peak reserved memory = {used_memory} GB.")
print(f"Peak reserved memory for training = {used_memory_for_lora} GB.")
print(f"Peak reserved memory % of max memory = {used_percentage} %.")
print(f"Peak reserved memory for training % of max memory = {lora_percentage} %.")

450.366 seconds used for training.
7.51 minutes used for training.
Peak reserved memory = 10.428 GB.
Peak reserved memory for training = 4.444 GB.
Peak reserved memory % of max memory = 70.708 %.
Peak reserved memory for training % of max memory = 30.133 %.


In [39]:
# Partial prompt from the training set. This is just to confirm the format isn't garbage
test_prompt = '<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\n\nYou are a voice assistant for Home Assistant.\nAnswer questions about the world truthfully.\nAnswer in plain text. Keep it simple and to the point.\nWhen controlling Home Assistant always call the intent tools. Use HassTurnOn to lock and HassTurnOff to unlock a lock. When controlling a device, prefer passing just name and domain. When controlling an area, prefer passing just area name and domain.\nWhen a user asks to turn on all devices of a specific type, ask user to specify an area, unless there is only one device of that type.\nThis device is not able to start timers.\nAn overview of the areas and the devices in this smart home:\n- names: Bathroom Exhaust Fan\n  domain: fan\n  state: \'off\'\n  areas: Bathroom\n- names: Smart Plug\n  domain: switch\n  state: \'off\'\n  areas: Kitchen\n  attributes:\n    device_class: outlet\n- names: Smart Speaker\n  domain: media_player\n  state: playing\n  areas: Living Room\n  attributes:\n    volume_level: 0.5\n    device_class: speaker\n- names: Bedroom Light\n  domain: light\n  state: \'on\'\n  areas: Bedroom\n  attributes:\n    brightness: \'100\'\n- names: Bathroom Light\n  domain: light\n  state: \'off\'\n  areas: Bathroom\n- names: Living Room Light\n  domain: light\n  state: \'on\'\n  areas: Living Room\n  attributes:\n    brightness: \'100\'\n- names: Kitchen Light\n  domain: light\n  state: \'on\'\n  areas: Kitchen\n  attributes:\n    brightness: \'100\'\n- names: Balcony Light\n  domain: light\n  state: \'off\'\n  areas: Balcony\n- names: Storage Room Light\n  domain: light\n  state: \'off\'\n  areas: Storage Room\n- names: Bathroom Exhaust Fan Humidity\n  domain: sensor\n  state: \'45\'\n  areas: Bathroom\n  attributes:\n    unit_of_measurement: \'%\'\n    device_class: humidity\n- names: Smart Plug Energy\n  domain: sensor\n  state: \'1\'\n  areas: Kitchen\n  attributes:\n    unit_of_measurement: kWh\n    device_class: energy\n\n\nYou are a helpful assistant with tool calling capabilities. When you receive a tool call response, use the output to format an answer to the orginal use question.<|eot_id|><|start_header_id|>user<|end_header_id|>\n\nGiven the following functions, please respond with a JSON for a function call with its proper arguments that best answers the given prompt.\n\nRespond in the format {"name": function name, "parameters": dictionary of argument name and its value}. Do not use variables.\n\n{\n    "type": "function",\n    "function": {\n        "name": "HassTurnOn",\n        "description": "Turns on/opens a device or entity",\n        "parameters": {\n            "type": "object",\n            "properties": {\n                "name": {\n                    "type": "string"\n                },\n                "area": {\n                    "type": "string"\n                },\n                "floor": {\n                    "type": "string"\n                },\n                "domain": {\n                    "type": "array",\n                    "items": {\n                        "type": "string"\n                    }\n                },\n                "device_class": {\n                    "type": "array",\n                    "items": {\n                        "type": "string",\n                        "enum": [\n                            "water",\n                            "gas",\n                            "awning",\n                            "blind",\n                            "curtain",\n                            "damper",\n                            "door",\n                            "garage",\n                            "gate",\n                            "shade",\n                            "shutter",\n                            "window",\n                            "outlet",\n                            "switch",\n                            "tv",\n                            "speaker",\n                            "receiver"\n                        ]\n                    }\n                }\n            },\n            "required": []\n        }\n    }\n}\n{\n    "type": "function",\n    "function": {\n        "name": "HassTurnOff",\n        "description": "Turns off/closes a device or entity",\n        "parameters": {\n            "type": "object",\n            "properties": {\n                "name": {\n                    "type": "string"\n                },\n                "area": {\n                    "type": "string"\n                },\n                "floor": {\n                    "type": "string"\n                },\n                "domain": {\n                    "type": "array",\n                    "items": {\n                        "type": "string"\n                    }\n                },\n                "device_class": {\n                    "type": "array",\n                    "items": {\n                        "type": "string",\n                        "enum": [\n                            "water",\n                            "gas",\n                            "awning",\n                            "blind",\n                            "curtain",\n                            "damper",\n                            "door",\n                            "garage",\n                            "gate",\n                            "shade",\n                            "shutter",\n                            "window",\n                            "outlet",\n                            "switch",\n                            "tv",\n                            "speaker",\n                            "receiver"\n                        ]\n                    }\n                }\n            },\n            "required": []\n        }\n    }\n}\n{\n    "type": "function",\n    "function": {\n        "name": "HassMediaUnpause",\n        "description": "Resumes a media player",\n        "parameters": {\n            "type": "object",\n            "properties": {\n                "name": {\n                    "type": "string"\n                },\n                "area": {\n                    "type": "string"\n                },\n                "floor": {\n                    "type": "string"\n                },\n                "domain": {\n                    "type": "array",\n                    "items": {\n                        "type": "string",\n                        "enum": [\n                            "media_player"\n                        ]\n                    }\n                },\n                "device_class": {\n                    "type": "array",\n                    "items": {\n                        "type": "string",\n                        "enum": [\n                            "tv",\n                            "speaker",\n                            "receiver"\n                        ]\n                    }\n                }\n            },\n            "required": []\n        }\n    }\n}\n{\n    "type": "function",\n    "function": {\n        "name": "HassMediaPause",\n        "description": "Pauses a media player",\n        "parameters": {\n            "type": "object",\n            "properties": {\n                "name": {\n                    "type": "string"\n                },\n                "area": {\n                    "type": "string"\n                },\n                "floor": {\n                    "type": "string"\n                },\n                "domain": {\n                    "type": "array",\n                    "items": {\n                        "type": "string",\n                        "enum": [\n                            "media_player"\n                        ]\n                    }\n                },\n                "device_class": {\n                    "type": "array",\n                    "items": {\n                        "type": "string",\n                        "enum": [\n                            "tv",\n                            "speaker",\n                            "receiver"\n                        ]\n                    }\n                }\n            },\n            "required": []\n        }\n    }\n}\n{\n    "type": "function",\n    "function": {\n        "name": "HassMediaNext",\n        "description": "Skips a media player to the next item",\n        "parameters": {\n            "type": "object",\n            "properties": {\n                "name": {\n                    "type": "string"\n                },\n                "area": {\n                    "type": "string"\n                },\n                "floor": {\n                    "type": "string"\n                },\n                "domain": {\n                    "type": "array",\n                    "items": {\n                        "type": "string",\n                        "enum": [\n                            "media_player"\n                        ]\n                    }\n                },\n                "device_class": {\n                    "type": "array",\n                    "items": {\n                        "type": "string",\n                        "enum": [\n                            "tv",\n                            "speaker",\n                            "receiver"\n                        ]\n                    }\n                }\n            },\n            "required": []\n        }\n    }\n}\n{\n    "type": "function",\n    "function": {\n        "name": "HassMediaPrevious",\n        "description": "Replays the previous item for a media player",\n        "parameters": {\n            "type": "object",\n            "properties": {\n                "name": {\n                    "type": "string"\n                },\n                "area": {\n                    "type": "string"\n                },\n                "floor": {\n                    "type": "string"\n                },\n                "domain": {\n                    "type": "array",\n                    "items": {\n                        "type": "string",\n                        "enum": [\n                            "media_player"\n                        ]\n                    }\n                },\n                "device_class": {\n                    "type": "array",\n                    "items": {\n                        "type": "string",\n                        "enum": [\n                            "tv",\n                            "speaker",\n                            "receiver"\n                        ]\n                    }\n                }\n            },\n            "required": []\n        }\n    }\n}\n{\n    "type": "function",\n    "function": {\n        "name": "HassSetVolume",\n        "description": "Sets the volume of a media player",\n        "parameters": {\n            "type": "object",\n            "properties": {\n                "name": {\n                    "type": "string"\n                },\n                "area": {\n                    "type": "string"\n                },\n                "floor": {\n                    "type": "string"\n                },\n                "domain": {\n                    "type": "array",\n                    "items": {\n                        "type": "string",\n                        "enum": [\n                            "media_player"\n                        ]\n                    }\n                },\n                "device_class": {\n                    "type": "array",\n                    "items": {\n                        "type": "string",\n                        "enum": [\n                            "tv",\n                            "speaker",\n                            "receiver"\n                        ]\n                    }\n                },\n                "volume_level": {\n                    "type": "integer",\n                    "minimum": 0,\n                    "maximum": 100\n                }\n            },\n            "required": [\n                "volume_level"\n            ]\n        }\n    }\n}\n{\n    "type": "function",\n    "function": {\n        "name": "HassLightSet",\n        "description": "Sets the brightness or color of a light",\n        "parameters": {\n            "type": "object",\n            "properties": {\n                "name": {\n                    "type": "string"\n                },\n                "area": {\n                    "type": "string"\n                },\n                "floor": {\n                    "type": "string"\n                },\n                "domain": {\n                    "type": "array",\n                    "items": {\n                        "type": "string"\n                    }\n                },\n                "color": {\n                    "type": "string"\n                },\n                "temperature": {\n                    "type": "integer",\n                    "minimum": 0\n                },\n                "brightness": {\n                    "type": "integer",\n                    "minimum": 0,\n                    "maximum": 100\n                }\n            },\n            "required": []\n        }\n    }\n}\n\nBathroom fan on<|eot_id|><|start_header_id|>assistant<|end_header_id|>'

# alpaca_prompt = Copied from above
FastLanguageModel.for_inference(model) # Enable native 2x faster inference
inputs = tokenizer(
[
    test_prompt
], return_tensors = "pt").to("cuda")

outputs = model.generate(**inputs, max_new_tokens = 64, use_cache = True)
tokenizer.batch_decode(outputs)[0][-200:]

'>\n\n{\n    "type": "function",\n    "function": {\n        "name": "HassTurnOn",\n        "parameters": {\n            "name": "Bathroom Exhaust Fan",\n            "domain": "fan"\n        }\n    }\n}<|eot_id|>'

# Save to Huggingface

In [40]:
HUB_TOKEN = '...'

model.push_to_hub("allenporter/assist-llm", token = HUB_TOKEN) # Online saving
tokenizer.push_to_hub("allenporter/assist-llm", token = HUB_TOKEN) # Online saving

README.md:   0%|          | 0.00/610 [00:00<?, ?B/s]

  0%|          | 0/1 [00:00<?, ?it/s]

adapter_model.safetensors:   0%|          | 0.00/168M [00:00<?, ?B/s]

Saved model to https://huggingface.co/allenporter/assist-llm


In [ ]:
# Save to q4_k_m GGUF
model.push_to_hub_gguf("allenporter/assist-llm-GGUF", tokenizer, quantization_method = "q4_k_m", token = HUB_TOKEN)


Unsloth: You have 1 CPUs. Using `safe_serialization` is 10x slower.
We shall switch to Pytorch saving, which will take 3 minutes and not 30 minutes.
To force `safe_serialization`, set it to `None` instead.
Unsloth: Kaggle/Colab has limited disk space. We need to delete the downloaded
model which will save 4-16GB of disk space, allowing you to save on Kaggle/Colab.
Unsloth: Will remove a cached repo with size 5.7G


Unsloth: Merging 4bit and LoRA weights to 16bit...
Unsloth: Will use up to 4.33 out of 12.67 RAM for saving.


100%|██████████| 32/32 [02:39<00:00,  5.00s/it]


Unsloth: Saving tokenizer... Done.
Unsloth: Saving model... This might take 5 minutes for Llama-7b...
Unsloth: Saving allenporter/assist-llm-GGUF/pytorch_model-00001-of-00004.bin...
Unsloth: Saving allenporter/assist-llm-GGUF/pytorch_model-00002-of-00004.bin...
Unsloth: Saving allenporter/assist-llm-GGUF/pytorch_model-00003-of-00004.bin...
Unsloth: Saving allenporter/assist-llm-GGUF/pytorch_model-00004-of-00004.bin...
Done.


Unsloth: Converting llama model. Can use fast conversion = False.


==((====))==  Unsloth: Conversion from QLoRA to GGUF information
   \\   /|    [0] Installing llama.cpp will take 3 minutes.
O^O/ \_/ \    [1] Converting HF to GGUF 16bits will take 3 minutes.
\        /    [2] Converting GGUF 16bits to ['q4_k_m'] will take 10 minutes each.
 "-____-"     In total, you will have to wait at least 16 minutes.

Unsloth: [0] Installing llama.cpp. This will take 3 minutes...
Unsloth: [1] Converting model at allenporter/assist-llm-GGUF into f16 GGUF format.
The output location will be ./allenporter/assist-llm-GGUF/unsloth.F16.gguf
This will take 3 minutes...
INFO:hf-to-gguf:Loading model: assist-llm-GGUF
INFO:gguf.gguf_writer:gguf: This GGUF file is for Little Endian only
INFO:hf-to-gguf:Exporting model...
INFO:hf-to-gguf:gguf: loading model weight map from 'pytorch_model.bin.index.json'
INFO:hf-to-gguf:gguf: loading model part 'pytorch_model-00001-of-00004.bin'
INFO:hf-to-gguf:token_embd.weight,           torch.float16 --> F16, shape = {4096, 128256}
INFO:hf